In [29]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import pandas_datareader as web
from pandas.plotting import lag_plot
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from datetime import datetime
import yfinance as yf
import pandas_datareader.data as pdr
import statsmodels.api as smapi
print(yf.pdr_override())

None


In [30]:
import warnings
warnings.filterwarnings('ignore')

In [31]:
data = pd.read_csv("Train_Status.csv")

In [32]:
data.columns

Index(['S No', 'Name of Stock'], dtype='object')

In [33]:
def smape_kun(y_true, y_pred):
    return np.mean((np.abs(y_pred - y_true) * 200/ (np.abs(y_pred) + np.abs(y_true))))

def arima_train(data,test,predict):
    predict = int(0.8 * data.shape[0])
    train_data, test_data = data[:-(predict)], data[-predict:]
    
    train_ar = train_data['open'].values
    test_ar = test_data['open'].values
    history = [x for x in train_ar]
    predictions = list()
    for t in range(len(test_ar)):
        model = ARIMA(history, order=(5,1,0))
        model_fit = model.fit()
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        obs = test_ar[t]
        history.append(obs)
    
    error = mean_squared_error(test_ar, predictions)
    error2 = smape_kun(test_ar, predictions)
    print('Testing Mean Squared Error: %.3f' % error)
    print('Symmetric mean absolute percentage error: %.3f' % error2)
    test_data['outcome']=predictions
    return test_data,error,error2

In [34]:
data.columns

Index(['S No', 'Name of Stock'], dtype='object')

In [35]:
out = []

In [36]:
already = []
# ["AAPL",
# "MSFT",
# "AMZN",
# "GOOGL",
# "BRK.B",
# "UNH",
# "TSLA",
# "GOOG",
# "XOM",
# "JNJ",
# "JPM",
# "NVDA",
# "PG",
# "V",
# "CVX",
# "HD",
# "LLY",
# "MA",
# "BAC",
# "PFE",
# "ABBV",
# "MRK",
# "META",
# "PEP",
# "KO",
# "COST",
# "AVGO",
# "TMO",
# "MCD",
# "WMT",
# "CSCO",
# "ABT",
# "WFC",
# "ACN"]

In [37]:
for i in data["Name of Stock"]:
        print("Running ARIMA for :"+i)
        if i not in already:
            try:
                data = pd.read_csv("individual_stocks_5yr/"+i+"_data.csv")
            except:
                start = datetime.strptime('2014-01-01', '%Y-%m-%d')
                end = datetime.strptime('2022-06-01', '%Y-%m-%d')
                data = pdr.get_data_yahoo(i, start, end).reset_index()
                data['DATE'] = data['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
                data.columns = ["date","open","high","low","close",'Adj Close',"volume",'DATE']
            test,predict = 180,12
            try:
                predict_data,error,error2 = arima_train(data,test,predict)
            except:
                continue
            predict_data.to_csv("results/"+i+"_results.csv")
    #         predict_data.outcome = predict_data.outcome.apply(lambda x : round(x,2))
    #         amount = 10000
    #         for k in predict_data.iterrows():
    #             k = k[1]
    #             if k['high']<k['outcome']<k['close']:
    #                 result = (k['outcome']-k['open'])/k['open']
    #             else:
    #                 result = (k['close']-k['open'])/k['open']
    #             amount = (amount*(100+result))/100
    #         total_profit = amount - 10000
    #         print(total_profit)
            out.append([i,error,error2])


Running ARIMA for :AAPL
Running ARIMA for :MSFT
Running ARIMA for :AMZN
Running ARIMA for :GOOGL
Running ARIMA for :BRK.B
Running ARIMA for :UNH
Running ARIMA for :TSLA
Running ARIMA for :GOOG
Running ARIMA for :XOM
Running ARIMA for :JNJ
Running ARIMA for :JPM
Running ARIMA for :NVDA
Running ARIMA for :PG
Running ARIMA for :V
Running ARIMA for :CVX
Running ARIMA for :HD
Running ARIMA for :LLY
Running ARIMA for :MA
Running ARIMA for :BAC
Running ARIMA for :PFE
Running ARIMA for :ABBV
Running ARIMA for :MRK
Running ARIMA for :META
Running ARIMA for :PEP
Running ARIMA for :KO
Running ARIMA for :COST
Running ARIMA for :AVGO
Running ARIMA for :TMO
Running ARIMA for :MCD
Running ARIMA for :WMT
Running ARIMA for :CSCO
Running ARIMA for :ABT
Running ARIMA for :WFC
Running ARIMA for :ACN
Running ARIMA for :DHR
Running ARIMA for :BMY
Testing Mean Squared Error: 0.913
Symmetric mean absolute percentage error: 1.027
Running ARIMA for :DIS
Testing Mean Squared Error: 1.476
Symmetric mean absolute 

In [38]:
final_data = pd.DataFrame(columns=["Name","MSE","MAPE"])#,"Total Profit"])

In [39]:
for i,x in enumerate(out):
    final_data.loc[i]=x

In [40]:
final_data.to_csv("final_results.csv")

In [26]:
final_data['Total Profit'].sum()

-70.46037204076856

In [41]:
final_data

,Name,MSE,MAPE
0,BMY,0.913348,1.027050
1,DIS,1.476159,0.832087
2,COP,1.125577,1.438312
3,LIN,9.655961,1.077956
4,NEE,1.476863,0.780152
5,VZ,0.251899,0.766363
6,TXN,0.831313,1.006272
7,AMGN,5.871194,1.135842
8,CRM,1.634342,1.230486
9,ADBE,2.715462,1.088267
